In [1]:
import numpy as np
import weakref
import contextlib

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [3]:
def as_tuple(x):
    if not isinstance(x,tuple):
        return (x,)
    return x

In [4]:
def as_variable(obj):
    if isinstance(obj,Variable):
        return obj
    return Variable(obj)

In [5]:
class Config:
    enable_backprop = True

In [6]:
@contextlib.contextmanager
def using_config(name,value):
    old_value = getattr(Config,name)
    setattr(Config,name,value)
    try:
        yield
    finally:
        setattr(Config,name,old_value)

In [7]:
def no_grad():
    return using_config('enable_backprop',False)

In [8]:
class Variable:
    __array_priotity__ = 200        # 演算子の優先度
    def __init__(self,data,name=None):
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError ('type:',type(data),'is not supported')
        self.data = data       # 変数の値の格納場所
        self.name = name
        self.grad = None
        self.creator = None    # 変数の生成元の関数を保持しておく
        self.generation = 0
    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    

    @property
    def shape(self):
        return self.data.shape
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    def set_creator(self,func):
        self.creator = func
        self.generation = func.generation + 1 # 優先度
    
    def clearglad(self):
        self.grad = None
    
    def backward(self,retaion_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            print('grad is None. create',self.grad)
        funcs = []
        seen = set()
        def add_func(f):
            if f not in seen:
                funcs.append(f)
                seen.add(f)
                funcs.sort(key=lambda x : x.generation)
        add_func(self.creator)
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            gxs = as_tuple(gxs)
            print(f.name +".backward(",gys,") = ",gxs)
            if f.name == "Add":
                print(f.name + '\'s input is :',f.inputs[0].data ,"and",f.inputs[1].data)
            else:
                print(f.name + '\'s input is :',f.inputs[0].data)
            for x,gx in zip(f.inputs,gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                if x.creator is not None:        # (x.cretor is None == x が入力変数) なのでそこでストップする
                    add_func(x.creator)
            if not retaion_grad:
                for y in f.outputs:
                    y().grad = None              # メモリを解放,yは弱参照なのでアクセスには()が必要

In [9]:
class Function:
    def __call__(self,*inputs:Variable) -> Variable:
        inputs = [as_variable(x) for x in inputs]
        xs = [x.data for x in inputs]
        ys = as_tuple(self.forward(*xs))
        outputs = [Variable(as_array(y)) for y in ys]
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])    # 入力変数のうち最大の世代数を採用する
            for output in outputs:
                output.set_creator(self)
        self.inputs = inputs    # 入力変数を保持しておく
        self.outputs = [weakref.ref(output) for output in outputs] # 弱参照
        return outputs if len(outputs) > 1 else outputs[0]   
    
    def forward(self,xs):
        raise NotImplementedError()
    
    def backward(self,gys):
        raise NotImplementedError()

In [10]:
class Add(Function):
    name = "Add"
    def forward(self,x0,x1):
        return x0 + x1
    
    def backward(self,gy):
        return gy,gy

def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)

Variable.__add__ = add
Variable.__radd__ = add

In [11]:
class Sub(Function):
    name = "Sub"
    def forward(self,x0,x1):
        return  x0 - x1
    
    def backward(self,gy):
        return gy, -gy
    
def sub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x0,x1)


def rsub(x0,x1):           
    x1 = as_array(x1)
    return Sub()(x1,x0)

Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [12]:
class Neg(Function):
    name = "Neg"
    def forward(self,x):
        return -x
    def backward(self,gy):
        return -gy

def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [13]:
class Mul(Function):
    name = "Mul"
    def forward(self,x0,x1):
        return x0 * x1
    
    def backward(self,gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0,x1):
    x1 = as_array(x1)
    return Mul()(x0,x1)


Variable.__mul__ = mul
Variable.__rmul__ = mul

In [14]:
class Div(Function):
    name = "Div"
    def forward(self,x0,x1):
        return x0 / x1
    
    def backward(self,gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1  = gy * (-x0 / x1 ** 2)
        return gx0,gx1
    
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div(x1,x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [15]:
class Pow(Function):
    name = "Pow"
    def __init__(self,c):
        self.c = c
    
    def forward(self,x):
        return x ** self.c
    
    def backward(selg,gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        
def pow(x,c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [16]:
class EXP(Function):
    name = "EXP"
    def forward(self,x):
        return np.exp(x)
    
    def backward(self,gy):
        return np.exp(self.inputs[0].data) * gy
    
    
def exp(x):
    return exp()(x)

In [17]:
class Square(Function):
    name = "Square"
    def forward(self,x):
        return x ** 2
    
    def backward(self,gy):
        return 2 * self.inputs[0].data * gy 

    
def square(x):
    return Square()(x)

In [18]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
z = add(square(x),square(y))
z.backward()

grad is None. create 1.0
Add.backward( [array(1.)] ) =  (array(1.), array(1.))
Add's input is : 4.0 and 9.0
Square.backward( [array(1.)] ) =  (6.0,)
Square's input is : 3.0
Square.backward( [array(1.)] ) =  (4.0,)
Square's input is : 2.0


In [19]:
print(x.grad)
print(y.grad)
print(z.data)

4.0
6.0
13.0


In [20]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a),square(a))
y.backward()

grad is None. create 1.0
Add.backward( [array(1.)] ) =  (array(1.), array(1.))
Add's input is : 16.0 and 16.0
Square.backward( [array(1.)] ) =  (8.0,)
Square's input is : 4.0
Square.backward( [array(1.)] ) =  (8.0,)
Square's input is : 4.0
Square.backward( [16.0] ) =  (64.0,)
Square's input is : 2.0


In [21]:
print(y.data)
print(x.grad)

32.0
64.0


In [22]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [23]:
x = Variable(np.array(2.0))

In [24]:
x + 3

variable(5.0)

In [25]:
-x

variable(-2.0)

In [26]:
2  - x

variable(0.0)

In [27]:
2 * x

variable(4.0)

In [28]:
x / 2

variable(1.0)

In [29]:
x ** 2

variable(4.0)